In [1]:
import pandas as pd
import numpy as np
import json
import os
import random
import string
import re
import matplotlib.pyplot as plt
import pickle

from pathlib import Path
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from catboost import CatBoostRegressor
from catboost import Pool
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error,mean_squared_log_error
import optuna.integration.lightgbm as lgbo
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import optuna
import umap
pd.set_option('display.max_columns', 2000)

SEED = 777



In [2]:
df_train = pd.read_pickle('../data/train/full_comp.pickle')
df_train.shape


(851524, 1387)

In [3]:
dorop_cols = []
for col in df_train.columns:
    if '_x' in col:
        dorop_cols.append(col)
len(dorop_cols), dorop_cols
df_train = df_train.drop(dorop_cols, axis=1)


In [4]:
df_train.shape

(851524, 1324)

In [5]:
df_pre_train = df_train[df_train['cites'].isnull()].reset_index(drop=True).sample(frac=0.4, random_state=777)
df_pre_valid = df_train[df_train['cites'].isnull()==False].reset_index(drop=True)
df_pre_train.shape, df_pre_valid.shape


((334563, 1324), (15117, 1324))

In [7]:
del_col = []
for col in df_train.columns:
    if 'doi_cites' in col:
        del_col.append(col)


In [8]:
df_train.head(1)

,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,authors_parsed,doi_cites,cites,doi_id,pub_publisher,pub_journals,pub_dois,update_date_y,first_created_date,last_created_date,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_first,author_num,category_main_detail,category_main,cs_y,econ_y,eess_y,math_y,nlin_y,physics_y,stat_y,category_name_parent_main_unique,category_name_parent_unique,category_name_unique,acc-phys_y,adap-org_y,alg-geom_y,ao-sci_y,astro-ph_y,atom-ph_y,bayes-an_y,chao-dyn_y,chem-ph_y,cmp-lg_y,comp-gas_y,cond-mat_y,dg-ga_y,funct-an_y,gr-qc_y,hep-ex_y,hep-lat_y,hep-ph_y,hep-th_y,math-ph_y,mtrl-th_y,nucl-ex_y,nucl-th_y,patt-sol_y,plasm-ph_y,q-alg_y,q-bio_y,q-fin_y,quant-ph_y,solv-int_y,supr-con_y,acc_y,adap_y,alg_y,ao_y,astro_y,atom_y,bayes_y,chao_y,chem_y,cmp_y,comp_y,cond_y,cs,dg_y,econ,eess,funct_y,gr_y,hep_y,math,mtrl_y,nlin,nucl_y,patt_y,physics,plasm_y,q_y,quant_y,solv_y,stat,supr_y,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,category_main_detail_label,category_name_parent_label,category_name_parent_main_label,category_name_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_label,doi_cites_q25_doi_id_label,doi_cites_q75_doi_id_label,doi_cites_mean_pub_publisher_label,doi_cites_count_pub_publisher_label,doi_cites_sum_pub_publisher_label,doi_cites_min_pub_publisher_label,doi_cites_max_pub_publisher_label,doi_cites_median_pub_publisher_label,doi_cites_std_pub_publisher_label,doi_cites_q10_pub_publisher_label,doi_cites_q25_pub_publisher_label,doi_cites_q75_pub_publisher_label,doi_cites_mean_submitter_label,doi_cites_count_submitter_label,doi_cites_sum_submitter_label,doi_cites_min_submitter_label,doi_c

In [ ]:
df_train.shape

In [10]:
df_train.submitter_label.nunique()

196215

In [11]:
# 目的変数
target = 'doi_cites'

y_train = df_pre_train[target].values
y_valid = df_pre_valid[target].values

x_train = df_pre_train.copy()
x_valid = df_pre_valid.copy()

x_train = x_train.drop(
    ['id', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique'], axis=1
)

x_valid = x_valid.drop(
    ['id', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique'], axis=1
)

# drop
x_train = x_train.drop(del_col, axis=1)
x_valid = x_valid.drop(del_col, axis=1)


params = {
    'objective': 'regression',
    'metric': 'rmse',
    #'learning_rate': 0.02,
    'num_iterations': 10000,
    'early_stopping_rounds': 50
}
trains = lgbo.Dataset(x_train, y_train)
valids = lgbo.Dataset(x_valid, y_valid)

best = lgbo.train(params, trains, valid_sets=valids,
                    verbose_eval=False,
                    num_boost_round=100,
                    early_stopping_rounds=50)
best

 Start training from score 2.134370
feature_fraction_stage2, val_score: 0.953546:  83%|########3 | 5/6 [05:30<01:04, 64.75s/it][LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.670089 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 242236
[LightGBM] [Info] Number of data points in the train set: 334563, number of used features: 1016
[LightGBM] [Info] Start training from score 2.134370
regularization_factors, val_score: 0.953546:   0%|          | 0/20 [00:00<?, ?it/s][LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.651522 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 242236
[LightGBM] [Info] Number of data points in the train set: 334563, number of used features: 1016
[LightGBM] [Info] Start training from score 2.134370
regularization_factors, val_score: 0.952694:   5%|5         | 1/20 [01:06<21:09, 66.80s/it

In [13]:
best.params

{'objective': 'regression',
 'metric': 'rmse',
 'feature_pre_filter': False,
 'lambda_l1': 3.2707572696344105e-08,
 'lambda_l2': 0.0002033018152732567,
 'num_leaves': 212,
 'feature_fraction': 0.5,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 50,
 'categorical_column': [241, 242, 243, 244, 245, 246, 248, 249, 250]}

In [15]:
df_pre_train = df_train[df_train['cites'].isnull()].reset_index(drop=True)
df_pre_valid = df_train[df_train['cites'].isnull()==False].reset_index(drop=True)
df_pre_train.shape, df_pre_valid.shape


((836407, 1324), (15117, 1324))

In [16]:
target = 'doi_cites'

y_train = df_pre_train[target].values
y_valid = df_pre_valid[target].values

x_train = df_pre_train.copy()
x_valid = df_pre_valid.copy()

x_train = x_train.drop(
    ['id', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique'], axis=1
)

x_valid = x_valid.drop(
    ['id', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique'], axis=1
)

# drop
x_train = x_train.drop(del_col, axis=1)
x_valid = x_valid.drop(del_col, axis=1)

lgb_params = {'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'feature_pre_filter': False,
    'lambda_l1': 3.2707572696344105e-08,
    'lambda_l2': 0.0002033018152732567,
    'num_leaves': 212,
    'feature_fraction': 0.5,
    'bagging_fraction': 1.0,
    'bagging_freq': 0,
    'min_child_samples': 20,
    'num_iterations': 10000,
    'early_stopping_rounds': 50,
}

# LightGBM
model = lgb.LGBMRegressor(**lgb_params)
model.fit(x_train, y_train,
            eval_set=(x_valid, y_valid),
            verbose=10,
            early_stopping_rounds=50
)

pred_doi_cites = model.predict(x_valid)
rmsle = mean_squared_error(y_valid, pred_doi_cites, squared=False)
print(f"rmsle: {rmsle}")

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/usr/local/lib/python3.8/dist-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))
Training until validation scores don't improve for 50 rounds
[10]	valid_0's rmse: 1.29575
[20]	valid_0's rmse: 1.25119
[30]	valid_0's rmse: 1.21271
[40

In [17]:
pickle.dump(model, open(f'../models/pred_doi_cites_lgb.pickle', 'wb'))

In [18]:
feature = x_train.columns
df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature, columns=['importance']).sort_values('importance', ascending=False)
df_feature.head(50)

,importance
author_first_label,8.681567e+06
update_date_unixtime,6.078218e+06
doi_id_label,5.961599e+06
submitter_label,5.736721e+06
update_date_days,5.349270e+06
update_ym,4.723300e+06
pub_publisher_label,4.413643e+06
category_name_label,4.211784e+06
author_num,2.114714e+06
category_name_parent_label,1.908630e+06


In [9]:
df_pre_train = df_train[df_train['cites'].isnull()].reset_index(drop=True)
df_pre_valid = df_train[df_train['cites'].isnull()==False].reset_index(drop=True)
df_pre_train.shape, df_pre_valid.shape

target = 'doi_cites'

y_train = df_pre_train[target].values
y_valid = df_pre_valid[target].values

x_train = df_pre_train.copy()
x_valid = df_pre_valid.copy()

x_train = x_train.drop(
    ['id', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique'], axis=1
)

x_valid = x_valid.drop(
    ['id', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'cites', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique'], axis=1
)

# drop
x_train = x_train.drop(del_col, axis=1)
x_valid = x_valid.drop(del_col, axis=1)


In [10]:
with open('../models/pred_doi_cites_lgb.pickle', mode='rb') as f:
    model = pickle.load(f)
model

LGBMRegressor(bagging_fraction=1.0, bagging_freq=0, early_stopping_rounds=50,
              feature_fraction=0.5, feature_pre_filter=False,
              lambda_l1=3.2707572696344105e-08, lambda_l2=0.0002033018152732567,
              learning_rate=0.01, metric='rmse', num_iterations=10000,
              num_leaves=212, objective='regression')

In [11]:
x_valid.shape

(15117, 1022)

In [12]:
pred_doi_cites = model.predict(x_valid)
rmsle = mean_squared_error(y_valid, pred_doi_cites, squared=False)
print(f"rmsle: {rmsle}")

rmsle: 0.9182986879892852


In [13]:
df_train = df_train[df_train['cites'].isnull() == False].reset_index(drop=True)
df_train['cites'] = np.log1p(df_train['cites'])
df_train['pred_doi_cites'] = pred_doi_cites
df_train.shape

(15117, 1325)

In [14]:
NFOLDS = 10
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=777)

df_train['fold_no'] = [0 for x in range(len(df_train))]
for fold, (train_idx, val_idx) in enumerate(kf.split(df_train)):
    df_train.loc[val_idx, 'fold_no'] = fold
df_train.head()


c_588  roberta_vec_589  roberta_vec_590  \
0         0.041590        12.282816         0.079741        -0.060993   
1         0.033566        12.149046         0.052788        -0.019711   
2        -0.036725        11.943683         0.009125        -0.021048   
3         0.084762        11.718389        -0.038374        -0.011906   
4         0.108661        11.651844        -0.063221        -0.013029   

   roberta_vec_591  roberta_vec_592  roberta_vec_593  roberta_vec_594  \
0         0.063440        -0.268049        -0.094870         0.024665   
1         0.077738        -0.249763        -0.025224         0.030602   
2         0.045698        -0.227533        -0.049069        -0.027898   
3        -0.001632        -0.224642        -0.079878        -0.099328   
4         0.032770        -0.256653         0.004424        -0.158540   

   roberta_vec_595  roberta_vec_596  roberta_vec_597  roberta_vec_598  \
0         0.001880         0.187180         0.040667         0.006068   
1        -0.058953         0.241092         0.107833        -0.050415   
2         0.021803         0.141909         0.143262         0.028343   
3         0.124719         0.179040         0.142606        -0.036361   
4         0.057062         0.279952         0.185350        -0.026612   

   roberta_vec_599  roberta_vec_600  roberta_vec_601  roberta_vec_602  \
0         0.023697         0.187934        -0.221596         0.068942   
1        -0.096370         0.116405        -0.205613         0.103472   
2         0.013603         0.211610        -0.150498        -0.003177   
3        -0.080203         0.120659        -0.049413         0.000616   
4        -0.073181         0.144206        -0.092292         0.073268   

   roberta_vec_603  roberta_vec_604  roberta_vec_605  roberta_vec_606  \
0         0.005736         0.098046         0.084538         0.091436   
1         0.020359         0.087780         0.165174         0.151956   
2         0.041957        -0.032630         0.211481         0.146104   
3         0.114762         0.136759         0.128589         0.083282   
4         0.087679         0.119163         0.097052         0.128711   

   roberta_vec_607  roberta_vec_608  roberta_vec_609  roberta_vec_610  \
0        -0.074916         0.082979         0.174427        -0.010615   
1        -0.155135         0.048719         0.136277         0.014289   
2        -0.185986         0.156548         0.081175         0.050995   
3        -0.119280         0.175465         0.175211         0.079239   
4        -0.163158         0.196666         0.096923         0.021693   

   roberta_vec_611  roberta_vec_612  roberta_vec_613  roberta_vec_614  \
0         0.956542        -0.157765         0.202592        -0.065107   
1         0.941260        -0.143702         0.072925        -0.135912   
2         0.899738        -0.161586         0.089926        -0.130796   
3         0.902776        -0.046504         0.255126        -0.002165   
4         0.768567        -0.057968         0.151456        -0.043386   

   roberta_vec_615  roberta_vec_616  roberta_vec_617  roberta_vec_618  \
0         0.086034         0.082509         0.116933        -0.130661   
1         0.126020         0.117271         0.201530        -0.107007   
2        -0.123190         0.086665         0.235653        -0.042723   
3        -0.024777         0.089408         0.198304         0.016981   
4        -0.004558         0.144182         0.187076         0.072016   

   roberta_vec_619  roberta_vec_620  roberta_vec_621  roberta_vec_622  \
0         0.056648        -0.001675        -0.010225         0.085153   
1         0.218389         0.065998        -0.083668         0.053277   
2         0.308070         0.091798        -0.021499        -0.014432   
3         0.230631         0.072909        -0.130362        -0.013177   
4         0.245870         0.112004        -0.171178         0.033259   

   roberta_vec_623  roberta_vec_624  roberta_vec_625  roberta_vec_626  \
0     

In [15]:
importance_0_cols = ['cs_y', 'econ_y', 'eess_y', 'math_y', 'nlin_y', 'physics_y',
       'stat_y', 'acc-phys_y', 'adap-org_y', 'alg-geom_y', 'ao-sci_y',
       'astro-ph_y', 'atom-ph_y', 'bayes-an_y', 'chao-dyn_y', 'chem-ph_y',
       'cmp-lg_y', 'comp-gas_y', 'cond-mat_y', 'dg-ga_y', 'funct-an_y',
       'gr-qc_y', 'math-ph_y', 'mtrl-th_y', 'nucl-th_y', 'patt-sol_y',
       'plasm-ph_y', 'q-alg_y', 'q-bio_y', 'q-fin_y', 'quant-ph_y',
       'solv-int_y', 'supr-con_y', 'acc_y', 'adap_y', 'alg_y', 'ao_y',
       'astro_y', 'atom_y', 'bayes_y', 'chao_y', 'chem_y', 'cmp_y',
       'comp_y', 'cond_y', 'cs', 'dg_y', 'econ', 'eess', 'funct_y',
       'gr_y', 'math', 'mtrl_y', 'nlin', 'patt_y', 'physics', 'plasm_y',
       'quant_y', 'solv_y', 'stat', 'supr_y', 'astro-ph.co',
       'astro-ph.ga', 'astro-ph.he', 'astro-ph.im', 'astro-ph.sr',
       'cond-mat.dis-nn', 'cond-mat.mes-hall', 'cond-mat.mtrl-sci',
       'cond-mat.other', 'cond-mat.soft', 'cond-mat.stat-mech', 'cs.ai',
       'cs.ar', 'cs.cc', 'cs.ce', 'cs.cg', 'cs.cl', 'cs.cr', 'cs.cv',
       'cs.cy', 'cs.db', 'cs.dc', 'cs.dl', 'cs.dm', 'cs.et', 'cs.fl',
       'cs.gl', 'cs.gr', 'cs.gt', 'cs.hc', 'cs.ir', 'cs.it', 'cs.lo',
       'cs.ma', 'cs.mm', 'cs.ms', 'cs.na', 'cs.ne', 'cs.ni', 'cs.oh',
       'cs.os', 'cs.pf', 'cs.pl', 'cs.ro', 'cs.sc', 'cs.sd', 'cs.se',
       'cs.si', 'cs.sy', 'econ.em', 'econ.gn', 'econ.th', 'eess.as',
       'eess.iv', 'eess.sp', 'eess.sy', 'math.ac', 'math.ap', 'math.at',
       'math.ca', 'math.co', 'math.ct', 'math.cv', 'math.dg', 'math.ds',
       'math.fa', 'math.gm', 'math.gn', 'math.gr', 'math.gt', 'math.ho',
       'math.it', 'math.kt', 'math.lo', 'math.mg', 'math.mp', 'math.na',
       'math.nt', 'math.oa', 'math.oc', 'math.pr', 'math.qa', 'math.ra',
       'math.rt', 'math.sg', 'math.sp', 'math.st', 'nlin.ao', 'nlin.cd',
       'nlin.cg', 'nlin.ps', 'nlin.si', 'physics.ao-ph', 'physics.app-ph',
       'physics.atm-clus', 'physics.bio-ph', 'physics.chem-ph',
       'physics.class-ph', 'physics.comp-ph', 'physics.data-an',
       'physics.ed-ph', 'physics.flu-dyn', 'physics.gen-ph',
       'physics.geo-ph', 'physics.hist-ph', 'physics.ins-det',
       'physics.med-ph', 'physics.optics', 'physics.plasm-ph',
       'physics.pop-ph', 'q-bio.bm', 'q-bio.cb', 'q-bio.gn', 'q-bio.mn',
       'q-bio.nc', 'q-bio.ot', 'q-bio.pe', 'q-bio.qm', 'q-bio.sc',
       'q-bio.to', 'q-fin.cp', 'q-fin.ec', 'q-fin.gn', 'q-fin.mf',
       'q-fin.pm', 'q-fin.pr', 'q-fin.rm', 'q-fin.st', 'q-fin.tr',
       'stat.ap', 'stat.co', 'stat.me', 'stat.ml', 'stat.ot', 'stat.th',
       'submitter_label', 'license_label',
       'doi_cites_min_author_first_label', 'doi_cites_min_doi_id_label',
       'doi_cites_q10_doi_id_label', 'doi_cites_q25_doi_id_label',
       'doi_cites_q75_doi_id_label', 'doi_cites_min_pub_publisher_label',
       'doi_cites_median_pub_publisher_label',
       'doi_cites_q10_pub_publisher_label',
       'doi_cites_q25_pub_publisher_label',
       'doi_cites_q75_pub_publisher_label', 'doi_cites_min_update_ym',
       'doi_cites_min_first_created_ym', 'doi_cites_count_license_label',
       'doi_cites_sum_license_label', 'doi_cites_min_license_label',
       'doi_cites_median_license_label', 'doi_cites_q10_license_label',
       'doi_cites_q25_license_label', 'doi_cites_q75_license_label',
       'doi_cites_mean_category_main_label',
       'doi_cites_min_category_main_label',
       'doi_cites_q10_category_main_label',
       'doi_cites_q25_category_main_label',
       'doi_cites_mean_category_main_detail_label',
       'doi_cites_count_category_main_detail_label',
       'doi_cites_sum_category_main_detail_label',
       'doi_cites_min_category_main_detail_label',
       'doi_cites_median_category_main_detail_label',
       'doi_cites_q10_category_main_detail_label',
       'doi_cites_q25_category_main_detail_label',
       'doi_cites_q75_category_main_detail_label',
       'doi_cites_min_category_name_parent_label',
       'doi_cites_q10_category_name_parent_label',
       'doi_cites_min_category_name_parent_main_label',
       'doi_cites_q10_category_name_parent_main_label',
       'doi_cites_q25_category_name_parent_main_label',
       'doi_cites_min_category_name_label',
       'diff_doi_cites_mean_submitter_label_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_doi_id_label_doi_cites_mean_pub_publisher_label',
       'diff_rate_doi_cites_mean_author_first_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_pub_publisher_label_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_license_label_doi_cites_mean_category_main_detail_label',
       'diff_doi_cites_mean_category_main_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_label_doi_cites_mean_category_main_detail_label',
       'diff_rate_doi_cites_mean_category_main_detail_label_doi_cites_mean_category_name_parent_main_label',
       'is_null_comments', 'is_null_journal-ref']


In [16]:
def make_statics_table(target, key, diff_key, x_train, x_valid, x_test):
    target_encoding = pd.concat([
        pd.DataFrame(x_train.groupby(key)[target].count()).rename(columns={target:f'te_{key}_{target}_count'}),
        pd.DataFrame(x_train.groupby(key)[target].min()).rename(columns={target:f'te_{key}_{target}_min'}),
        pd.DataFrame(x_train.groupby(key)[target].max()).rename(columns={target:f'te_{key}_{target}_max'}),
        pd.DataFrame(x_train.groupby(key)[target].std()).rename(columns={target:f'te_{key}_{target}_std'}),
        pd.DataFrame(x_train.groupby(key)[target].mean()).rename(columns={target:f'te_{key}_{target}_mean'}),
        pd.DataFrame(x_train.groupby(key)[target].median()).rename(columns={target:f'te_{key}_{target}_median'}),
    ], axis = 1)

    x_train = pd.merge(x_train, target_encoding, on=key, how='left')
    x_valid = pd.merge(x_valid, target_encoding, on=key, how='left')
    x_test = pd.merge(x_test, target_encoding, on=key, how='left')

    x_train[f'te_diff_{diff_key}_{key}_{target}_mean'] = x_train[diff_key] - x_train[f'te_{key}_{target}_mean']
    x_train[f'te_rate_{diff_key}_{key}_{target}_mean'] = x_train[diff_key] / x_train[f'te_{key}_{target}_mean']
    x_valid[f'te_diff_{diff_key}_{key}_{target}_mean'] = x_valid[diff_key] - x_valid[f'te_{key}_{target}_mean']
    x_valid[f'te_rate_{diff_key}_{key}_{target}_mean'] = x_valid[diff_key] / x_valid[f'te_{key}_{target}_mean']
    x_test[f'te_diff_{diff_key}_{key}_{target}_mean'] = x_test[diff_key] - x_test[f'te_{key}_{target}_mean']
    x_test[f'te_rate_{diff_key}_{key}_{target}_mean'] = x_test[diff_key] / x_test[f'te_{key}_{target}_mean']

    return x_train, x_valid, x_test

In [17]:
cat_columns = []
for k, v in zip(df_train.columns, df_train.dtypes):
    if 'label' in k and 'doi_cites' not in k  and 'submitter_label' not in k:
        cat_columns.append(k)
cat_columns

['doi_id_label',
 'author_first_label',
 'pub_publisher_label',
 'license_label',
 'category_main_label',
 'category_main_detail_label',
 'category_name_parent_label',
 'category_name_parent_main_label',
 'category_name_label']

In [34]:
# パラメーター調整
result_y = []
lgb_result_proba_test = []
lgb_result_proba_valid = []

fold_no = 2
test_fold_no = fold_no
valid_fold_no = fold_no + 1
if valid_fold_no == NFOLDS:
    valid_fold_no = 0

# train
train = df_train.copy()
y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
y_test = train[train['fold_no'] == test_fold_no]['cites'].values

train = train.drop(
    ['id', 'authors', 'title', 'comments',
    'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
    , axis=1
)
train = train.drop(importance_0_cols, axis=1)
train['diff_pred_doi_cites'] = train['pred_doi_cites'] - train['doi_cites']
train['rate_pred_doi_cites'] = train['pred_doi_cites'] / (train['doi_cites'] + 1)

x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
x_valid = train[train['fold_no'] == valid_fold_no]
x_test = train[train['fold_no'] == test_fold_no]

'''
# target encoding
target = 'doi_cites'
key = 'pred_doi_cites'
x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

keys = ['update_ym']
target = 'cites'
for key in keys:
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
'''

# drop
x_train = x_train.drop(['cites', 'fold_no'], axis=1)
x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
x_test = x_test.drop(['cites', 'fold_no'], axis=1)

params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'num_iterations': 10000,
    'early_stopping_rounds': 50
}
best_params, history = {}, []
trains = lgb.Dataset(x_train, y_train)
valids = lgb.Dataset(x_valid, y_valid)

best = lgbo.train(params, trains, valid_sets=valids,
                    verbose_eval=False,
                    num_boost_round=100,
                    early_stopping_rounds=50)
best.params

l Bins 250074
[LightGBM] [Info] Number of data points in the train set: 12093, number of used features: 1057
[LightGBM] [Info] Start training from score 2.576181



regularization_factors, val_score: 0.504419:  10%|#         | 2/20 [00:37<03:40, 12.23s/it]


regularization_factors, val_score: 0.504419:  15%|#5        | 3/20 [00:37<03:35, 12.66s/it][I 2021-03-27 10:12:41,185] Trial 45 finished with value: 0.505259168045804 and parameters: {'lambda_l1': 0.07433825739446001, 'lambda_l2': 1.961952345462962e-07}. Best is trial 45 with value: 0.505259168045804.



regularization_factors, val_score: 0.504419:  15%|#5        | 3/20 [00:37<03:35, 12.66s/it][LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.036980 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 250074
[LightGBM] [Info] Number of data points in the train set: 12093, number of used features: 1057
[LightGBM] [Info] Start training from score 2.57

{'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.01,
 'feature_pre_filter': False,
 'lambda_l1': 8.856362720452578e-07,
 'lambda_l2': 0.0002814044321572516,
 'num_leaves': 31,
 'feature_fraction': 0.42,
 'bagging_fraction': 0.8762325382386713,
 'bagging_freq': 4,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 50,
 'categorical_column': [46, 47, 48, 49, 51, 52, 53]}

In [84]:
result_y_test = []
result_y_valid = []
lgb_result_proba_test = []
lgb_result_proba_valid = []

lgb_params = {'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.01,
 'feature_pre_filter': False,
 'lambda_l1': 8.856362720452578e-07,
 'lambda_l2': 0.0002814044321572516,
 'num_leaves': 31,
 'feature_fraction': 0.42,
 'bagging_fraction': 0.8762325382386713,
 'bagging_freq': 4,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 50,}

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    train['diff_pred_doi_cites'] = train['pred_doi_cites'] - train['doi_cites']
    train['rate_pred_doi_cites'] = (train['pred_doi_cites'] + 1) - (train['doi_cites'] + 1)

    y_train = train[~train['fold_no'].isin([valid_fold_no])]['cites'].values # test_fold_no, 
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )

    train = train.drop(importance_0_cols, axis=1)

    x_train = train[~train['fold_no'].isin([valid_fold_no])] # test_fold_no, 
    x_valid = train[train['fold_no'] == valid_fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    '''
    # target encoding
    target = 'doi_cites'
    key = 'pred_doi_cites'
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

    keys = ['update_ym']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
    '''

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    #x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(x_train, y_train,
                eval_set=(x_valid, y_valid),
                eval_metric='rmse',
                verbose=100,
                early_stopping_rounds=50,
    )

    pickle.dump(model, open(f'../models/lgb_{fold_no}.pickle', 'wb'))

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    #lgb_result_proba_test.extend(fold_result_test)
    lgb_result_proba_valid.extend(fold_result_valid)
    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    #print(f"fold {fold_no} lgb score: {rmsle}")


print("valid", mean_squared_error(result_y_valid, lgb_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, lgb_result_proba_test, squared=False))



/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Warning] feature_fraction is set=0.42, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42
[LightGBM] [Warning] lambda_l1 is set=8.856362720452578e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.856362720452578e-07
[LightGBM] [Warning] bagging_fraction is set=0.8762325382386713, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8762325382386713
[LightGBM] [Warning] lambda_l2 is set=0.0002814044321572516, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.000281

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
[LightGBM] [Warning] feature_fraction is set=0.42, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42
[LightGBM] [Warning] lambda_l1 is set=8.856362720452578e-07, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.856362720452578e-07
[LightGBM] [Warning] bagging_fraction is set=0.8762325382386713, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8762325382386713
[LightGBM] [Warning] lambda_l2 is set=0.0002814044321572516, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.000281

In [24]:
best.params

{'objective': 'regression',
 'metric': 'rmse',
 'learning_rate': 0.02,
 'feature_pre_filter': False,
 'lambda_l1': 0.37475414935822066,
 'lambda_l2': 1.4093025694101272e-07,
 'num_leaves': 31,
 'feature_fraction': 0.8,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 20,
 'num_iterations': 2000,
 'early_stopping_round': 50,
 'categorical_column': [242, 243, 244, 245, 246, 247, 249, 250, 251]}

In [28]:
df_temp = pd.DataFrame(df_train.isna().sum()).rename(columns={0:'null_num'})
df_temp[df_temp['null_num'] > 0]

,null_num
submitter,132
comments,1943
journal-ref,5189
report-no,13230
license,4759
pub_publisher,110
pub_journals,110
pub_dois,110
doi_cites_std_author_first_label,673
doi_cites_std_doi_id_label,5


In [ ]:
from sklearn.ensemble import RandomForestRegressor
result_y_test = []
result_y_valid = []
#rf_result_proba_test = []
rf_result_proba_valid = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    train = train.fillna(0)

    y_train = train[~train['fold_no'].isin([valid_fold_no])]['cites'].values # test_fold_no, 
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    #y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )

    train = train.drop(importance_0_cols, axis=1)

    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    #x_test = train[train['fold_no'] == test_fold_no]

    '''
    # target encoding
    target = 'doi_cites'
    key = 'pred_doi_cites'
    x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)

    keys = ['update_ym']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, 'doi_cites', x_train, x_valid, x_test)
    '''

    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)

    model = RandomForestRegressor(n_estimators=24, criterion='mse', n_jobs=-1)
    model.fit(x_train, y_train)
    pickle.dump(model, open(f'../models/rf_{fold_no}.pickle', 'wb'))

    #fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    #result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    #rf_result_proba_test.extend(fold_result_test)
    rf_result_proba_valid.extend(fold_result_valid)
    #rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"fold {fold_no} rf score: {rmsle}")


print("valid", mean_squared_error(result_y_valid, rf_result_proba_valid, squared=False))
#print("test", mean_squared_error(result_y_test, rf_result_proba_test, squared=False))



fold 0 lgb score: 0.5067466282059222
fold 1 lgb score: 0.5070536799832732
fold 2 lgb score: 0.5487566432941057
fold 3 lgb score: 0.5383677383941933
fold 4 lgb score: 0.5168385937490483
fold 5 lgb score: 0.5184729250645402
fold 6 lgb score: 0.5322843342453957
fold 7 lgb score: 0.5063263130545729
fold 8 lgb score: 0.5133449767316445
fold 9 lgb score: 0.537059322815508
valid 0.5240871872446028
test 0.5227295515450585


In [74]:
def objective(trial):
    categorical_features_indices = np.where(x_train.dtypes == 'category')[0]

    train_pool = Pool(x_train, y_train, cat_features=categorical_features_indices)
    test_pool = Pool(x_test, y_test, cat_features=categorical_features_indices)
    
    # Parameters
    param = {
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.01, 0.1),
        "depth": trial.suggest_int("depth", 1, 12),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]
        ),
        "iterations": 1000,
        "learning_rate": 0.03,
        "loss_function": "RMSE",
        "eval_metric": "RMSE",
        "random_seed": SEED,
    }

    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)

    model = CatBoostRegressor(**param)
    model.fit(train_pool, eval_set=test_pool, verbose=200, early_stopping_rounds=20)
    
    preds = model.predict(test_pool)
    return mean_squared_error(y_test, preds, squared=False)

In [75]:
fold_no = 2
test_fold_no = fold_no
valid_fold_no = fold_no + 1
if valid_fold_no == NFOLDS:
    valid_fold_no = 0

# train
train = df_train.copy()
y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
y_test = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values

train = train.drop(
    ['id', 'authors', 'title', 'comments',
    'journal-ref', 'doi', 'report-no', 'categories', 'license',
    'abstract', 'versions', 'authors_parsed', 'pub_publisher',
    'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
    , axis=1
)
train = train.drop(importance_0_cols, axis=1)
train['diff_pred_doi_cites'] = train['pred_doi_cites'] - train['doi_cites']
train['rate_pred_doi_cites'] = train['pred_doi_cites'] / (train['doi_cites'] + 1)

x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
x_test = train[train['fold_no'].isin([test_fold_no, valid_fold_no])]

x_train = x_train.drop(['cites', 'fold_no'], axis=1)
x_test = x_test.drop(['cites', 'fold_no'], axis=1)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=30, timeout=600)


[I 2021-03-27 13:14:24,743] A new study created in memory with name: no-name-3c4a5d94-a509-479b-bb32-0534db732653
0:	learn: 1.1743888	test: 1.1671023	best: 1.1671023 (0)	total: 8.96ms	remaining: 8.95s
200:	learn: 0.5220326	test: 0.5447488	best: 0.5447488 (200)	total: 1.28s	remaining: 5.08s
400:	learn: 0.5033216	test: 0.5310702	best: 0.5310702 (400)	total: 2.41s	remaining: 3.6s
600:	learn: 0.4937408	test: 0.5264670	best: 0.5264670 (600)	total: 3.5s	remaining: 2.33s
800:	learn: 0.4871515	test: 0.5240054	best: 0.5240054 (800)	total: 4.64s	remaining: 1.15s
[I 2021-03-27 13:14:31,065] Trial 0 finished with value: 0.5227816675067647 and parameters: {'colsample_bylevel': 0.09178193753637694, 'depth': 3, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'subsample': 0.22889708929840308}. Best is trial 0 with value: 0.5227816675067647.
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.5227816692
bestIteration = 950

Shrink model to first 951 iterations.
0:	learn: 1.1701

In [76]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials: 15
Best trial:
  Value: 0.5125028838315051
  Params: 
    colsample_bylevel: 0.052922169991835896
    depth: 6
    boosting_type: Ordered
    bootstrap_type: MVS


In [81]:
SEED = 777
result_y_test = []
result_y_valid = []
cat_result_proba_test = []
cat_result_proba_valid = []

for fold_no in range(NFOLDS):
    test_fold_no = fold_no
    valid_fold_no = fold_no + 1
    if valid_fold_no == NFOLDS:
        valid_fold_no = 0

    # train
    train = df_train.copy()
    y_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]['cites'].values
    y_valid = train[train['fold_no'] == valid_fold_no]['cites'].values
    y_test = train[train['fold_no'] == test_fold_no]['cites'].values

    train = train.drop(
        ['id', 'authors', 'title', 'comments',
        'journal-ref', 'doi', 'report-no', 'categories', 'license',
        'abstract', 'versions', 'authors_parsed', 'pub_publisher',
        'update_date_y', 'first_created_date', 'last_created_date', 'doi_id', 'submitter', 'author_first', 'category_main', 'category_main_detail', 'category_name_parent_main_unique', 'category_name_parent_unique', 'category_name_unique']
        , axis=1
    )
    train = train.drop(importance_0_cols, axis=1)
    train['diff_pred_doi_cites'] = train['pred_doi_cites'] - train['doi_cites']
    train['rate_pred_doi_cites'] = train['pred_doi_cites'] / (train['doi_cites'] + 1)

    x_train = train[~train['fold_no'].isin([test_fold_no, valid_fold_no])]
    x_valid = train[train['fold_no'] == valid_fold_no]
    x_test = train[train['fold_no'] == test_fold_no]

    # target encoding
    #target = 'doi_cites'
    #key = 'pred_doi_cites'
    #x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)

    #keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label', 'category_name_label', 'pred_doi_cites']
    '''
    keys = ['update_ym', 'license_label', 'pub_publisher_label', 'category_main_label', 'category_name_parent_label', 'category_name_parent_main_label']
    target = 'cites'
    for key in keys:
        x_train, x_valid, x_test = make_statics_table(target, key, x_train, x_valid, x_test)
    '''
    
    # drop
    x_train = x_train.drop(['cites', 'fold_no'], axis=1)
    x_valid = x_valid.drop(['cites', 'fold_no'], axis=1)
    x_test = x_test.drop(['cites', 'fold_no'], axis=1)


    cat_features = np.where(x_train.dtypes == 'category')[0]
    train_pool = Pool(x_train, y_train, cat_features=cat_features)
    validate_pool = Pool(x_valid, y_valid, cat_features=cat_features)
    test_pool = Pool(x_test, y_test, cat_features=cat_features)
    
    model = CatBoostRegressor(
        loss_function='RMSE',
        eval_metric="RMSE",
        colsample_bylevel=0.052922169991835896,
        depth=6,
        boosting_type='Ordered',
        bootstrap_type='MVS',
        iterations=10000,
        random_seed=SEED,
    )
    #model = lgb.LGBMRegressor(**lgb_params)
    model.fit(train_pool,
                eval_set=validate_pool,
                verbose=100,
                early_stopping_rounds=20,
    )

    pickle.dump(model, open(f'../models/cat_{fold_no}.pickle', 'wb'))

    fold_result_test = model.predict(x_test)
    fold_result_valid = model.predict(x_valid)
    result_y_test.extend(y_test)
    result_y_valid.extend(y_valid)
    cat_result_proba_test.extend(fold_result_test)
    cat_result_proba_valid.extend(fold_result_valid)

    rmsle = mean_squared_error(y_test, fold_result_test, squared=False)
    print(f"fold {fold_no} lgb score: {rmsle}")

print("valid", mean_squared_error(result_y_valid, cat_result_proba_valid, squared=False))
print("test", mean_squared_error(result_y_test, cat_result_proba_test, squared=False))




Learning rate set to 0.017583
0:	learn: 1.1796004	test: 1.1766413	best: 1.1766413 (0)	total: 26.1ms	remaining: 4m 21s
100:	learn: 0.5916054	test: 0.5863165	best: 0.5863165 (100)	total: 3.5s	remaining: 5m 43s
200:	learn: 0.5259664	test: 0.5166985	best: 0.5166985 (200)	total: 6.71s	remaining: 5m 27s
300:	learn: 0.5097635	test: 0.5012702	best: 0.5012702 (300)	total: 9.92s	remaining: 5m 19s
400:	learn: 0.5005640	test: 0.4946617	best: 0.4946617 (400)	total: 13.2s	remaining: 5m 15s
500:	learn: 0.4936871	test: 0.4908874	best: 0.4908874 (500)	total: 16.4s	remaining: 5m 10s
600:	learn: 0.4877124	test: 0.4877624	best: 0.4877620 (598)	total: 19.6s	remaining: 5m 6s
700:	learn: 0.4815889	test: 0.4857584	best: 0.4857584 (700)	total: 22.8s	remaining: 5m 2s
800:	learn: 0.4758116	test: 0.4844378	best: 0.4844255 (798)	total: 26s	remaining: 4m 58s
900:	learn: 0.4700288	test: 0.4833306	best: 0.4833306 (900)	total: 29.4s	remaining: 4m 56s
Stopped by overfitting detector  (20 iterations wait)

bestTest = 0.

NameError: name 'result_y' is not defined

valid 0.49898602590195396
test 0.4992670981883152


In [53]:
df_result = pd.DataFrame([lgb_result_proba_test, cat_result_proba_test, result_y_test]).T.rename(columns={0:'lgb', 1:'cat', 2:'doi_cites', 3:'pred_doi_cites', 4:'y'})
df_result.head(50)

,lgb,cat,doi_cites
0,1.616322,1.668196,1.386294
1,1.150935,1.139835,3.663562
2,2.656537,2.606077,2.564949
3,2.070312,2.100779,1.791759
4,2.703382,2.741201,2.833213
5,2.786024,2.910353,2.995732
6,1.261919,1.377220,1.386294
7,1.120770,1.370008,1.609438
8,2.982970,2.812338,2.639057
9,1.282977,1.284003,1.386294


In [54]:
df_result.shape, len(result_y_test)

((15117, 3), 15117)

In [56]:


y = result_y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.25, random_state=0)
#model = linear_model.ElasticNet(alpha=0.7, l1_ratio=0.7, max_iter=5000)


In [57]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)
pred = model.predict(X_test)

mean_squared_error(y_test, pred, squared=False)

0.49743021455230063

In [58]:
lgb_params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.1,
    'feature_pre_filter': False,
    'num_leaves': 31,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'bagging_freq': 0,
    'min_child_samples': 20,
    'num_iterations': 2000,
    'early_stopping_round': 50,
}
model = lgb.LGBMRegressor(**lgb_params)
model.fit(X_train, y_train,
    eval_set=(X_test, y_test),
    eval_metric='rmse',
    verbose=100,
)

pred = model.predict(X_test)

mean_squared_error(y_test, pred, squared=False)

[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[34]	valid_0's rmse: 0.507305
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


0.5073053637597862

In [28]:
x_train.columns[categorical_features_indices]

Index(['submitter_label', 'doi_id_label', 'author_first_label',
       'pub_publisher_label', 'license_label', 'category_main_label',
       'category_name_parent_label', 'category_name_parent_main_label',
       'category_name_label'],
      dtype='object')

In [ ]:
 # 0.5029752173028096 lgb normal

In [33]:
x_train.head()

,doi_cites,pub_journals,pub_dois,update_year,first_created_year,last_created_year,update_month,first_created_month,last_created_month,update_ym,first_created_ym,last_created_ym,update_day,first_created_day,last_created_day,update_date_unixtime,first_created_unixtime,last_created_unixtime,diff_update_date_unixtime,diff_created_unixtime,num_created,update_date_days,first_created_days,last_created_days,diff_created_days,rate_created_days,author_num,astro-ph.co,astro-ph.ep,astro-ph.ga,astro-ph.he,astro-ph.im,astro-ph.sr,cond-mat.dis-nn,cond-mat.mes-hall,cond-mat.mtrl-sci,cond-mat.other,cond-mat.quant-gas,cond-mat.soft,cond-mat.stat-mech,cond-mat.str-el,cond-mat.supr-con,cs.ai,cs.ar,cs.cc,cs.ce,cs.cg,cs.cl,cs.cr,cs.cv,cs.cy,cs.db,cs.dc,cs.dl,cs.dm,cs.ds,cs.et,cs.fl,cs.gl,cs.gr,cs.gt,cs.hc,cs.ir,cs.it,cs.lg,cs.lo,cs.ma,cs.mm,cs.ms,cs.na,cs.ne,cs.ni,cs.oh,cs.os,cs.pf,cs.pl,cs.ro,cs.sc,cs.sd,cs.se,cs.si,cs.sy,econ.em,econ.gn,econ.th,eess.as,eess.iv,eess.sp,eess.sy,math.ac,math.ag,math.ap,math.at,math.ca,math.co,math.ct,math.cv,math.dg,math.ds,math.fa,math.gm,math.gn,math.gr,math.gt,math.ho,math.it,math.kt,math.lo,math.mg,math.mp,math.na,math.nt,math.oa,math.oc,math.pr,math.qa,math.ra,math.rt,math.sg,math.sp,math.st,nlin.ao,nlin.cd,nlin.cg,nlin.ps,nlin.si,physics.acc-ph,physics.ao-ph,physics.app-ph,physics.atm-clus,physics.atom-ph,physics.bio-ph,physics.chem-ph,physics.class-ph,physics.comp-ph,physics.data-an,physics.ed-ph,physics.flu-dyn,physics.gen-ph,physics.geo-ph,physics.hist-ph,physics.ins-det,physics.med-ph,physics.optics,physics.plasm-ph,physics.pop-ph,physics.soc-ph,physics.space-ph,q-bio.bm,q-bio.cb,q-bio.gn,q-bio.mn,q-bio.nc,q-bio.ot,q-bio.pe,q-bio.qm,q-bio.sc,q-bio.to,q-fin.cp,q-fin.ec,q-fin.gn,q-fin.mf,q-fin.pm,q-fin.pr,q-fin.rm,q-fin.st,q-fin.tr,stat.ap,stat.co,stat.me,stat.ml,stat.ot,stat.th,acc-phys,adap-org,alg-geom,ao-sci,astro-ph,atom-ph,bayes-an,chao-dyn,chem-ph,cmp-lg,comp-gas,cond-mat,dg-ga,funct-an,gr-qc,hep-ex,hep-lat,hep-ph,hep-th,math-ph,mtrl-th,nucl-ex,nucl-th,patt-sol,plasm-ph,q-alg,q-bio,q-fin,quant-ph,solv-int,supr-con,acc,adap,alg,ao,astro,atom,bayes,chao,chem,cmp,comp,cond,cs,dg,econ,eess,funct,gr,hep,math,mtrl,nlin,nucl,patt,physics,plasm,q,quant,solv,stat,supr,submitter_label,doi_id_label,author_first_label,pub_publisher_label,license_label,category_main_label,doi_cites_mean_author_first_label,doi_cites_count_author_first_label,doi_cites_sum_author_first_label,doi_cites_min_author_first_label,doi_cites_max_author_first_label,doi_cites_median_author_first_label,doi_cites_std_author_first_label,doi_cites_q10_author_first_label,doi_cites_q25_author_first_label,doi_cites_q75_author_first_label,doi_cites_mean_doi_id_label,doi_cites_count_doi_id_label,doi_cites_sum_doi_id_label,doi_cites_min_doi_id_label,doi_cites_max_doi_id_label,doi_cites_median_doi_id_label,doi_cites_std_doi_id_label,doi_cites_q10_doi_id_label,doi_cites_q25_doi_id_label,doi_cites_q75_doi_id_label,doi_cites_mean_pub_publisher_label,doi_cites_count_pub_publisher_label,doi_cites_sum_pub_publisher_label,doi_cites_min_pub_publisher_label,doi_cites_max_pub_publisher_label,doi_cites_median_pub_publisher_label,doi_cites_std_pub_publisher_label,doi_cites_q10_pub_publisher_label,doi_cites_q25_pub_publisher_label,doi_cites_q75_pub_publisher_label,doi_cites_mean_submitter_label,doi_cites_count_submitter_label,doi_cites_sum_submitter_label,doi_cites_min_submitter_label,doi_cites_max_submitter_label,doi_cites_median_submitter_label,doi_cites_std_submitter_label,doi_cites_q10_submitter_label,doi_cites_q25_submitter_label,doi_cites_q75_submitter_label,doi_cites_mean_category_main_label,doi_cites_count_category_main_label,doi_cites_sum_category_main_label,doi_cites_min_category_main_label,doi_cites_max_category_main_label,doi_cites_median_category_main_label,doi_cites_std_category_main_label,doi_cites_q10_category_main_label,doi_cites_q25_category_main_label,doi_cites_q75_category_main_label,doi_cites_mean_update_ym,doi_cites_count_update_ym,doi_cites_sum

In [45]:
feature = x_train.copy()
df_feature = pd.DataFrame(model.booster_.feature_importance(importance_type='gain'), index=feature.columns, columns=['importance']).sort_values('importance', ascending=False)
df_feature.head(50)


,importance
diff_doi_cites_doi_cites_mean_category_main_label,45452.756427
te_diff_license_label_cites_mean,9845.287023
diff_rate_doi_cites_doi_cites_mean_category_main_label,9263.424620
te_rate_license_label_cites_mean,3971.043035
diff_doi_cites_doi_cites_mean_doi_id_label,1151.217911
doi_cites,696.934825
diff_doi_cites_doi_cites_mean_update_ym,596.695166
last_created_ym,393.102887
diff_rate_doi_cites_doi_cites_mean_pub_publisher_label,359.453805
category_main_label,283.692540


In [14]:
df_feature.tail(400)


,importance
rbert_vec_raw_768_665,0.0
rbert_vec_raw_768_697,0.0
cs.ds,0.0
rbert_vec_raw_768_695,0.0
rbert_vec_raw_768_693,0.0
...,...
rbert_vec_raw_768_341,0.0
bayes-an,0.0
atom-ph,0.0
rbert_vec_raw_768_344,0.0
